# Portugal Energy System Model - PyPSA Analysis

**Group Q** - Data Science for Energy System Modeling (DSESM)

---

## Project Overview

This notebook implements a comprehensive PyPSA-based energy system model for Portugal. We analyze the electricity sector with focus on:
- Renewable energy integration (solar, wind, hydro)
- Network topology and constraints
- Optimal capacity expansion
- Policy scenarios and their impacts


**Team Members:**
- Avinash Varghese
- Sunder Shrestha

## 1. Setup and Configuration

Import required libraries and set up the environment.

In [1]:
# Importing libraries
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Geospatial libraries
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Atlite for renewable resource assessment
import atlite

# Configuration
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries imported successfully")
print(f"PyPSA version: {pypsa.__version__}")

c:\Users\nashm\anaconda3\envs\dsesm\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ All libraries imported successfully
PyPSA version: 0.33.2


### 1.1 Define Project Paths and Parameters

In [2]:
# Project directory structure
BASE_DIR = Path.cwd()
DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"
RESULTS_DIR = BASE_DIR / "results"
FIGURES_DIR = BASE_DIR / "figures"

# Create directories if they don't exist
for directory in [DATA_RAW, DATA_PROCESSED, RESULTS_DIR, FIGURES_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Model parameters
COUNTRY = "Portugal"
YEAR = 2024
SNAPSHOT_HOURS = 8784  # Full year hourly resolution (2024 is a leap year)
SOLVER = "gurobi"  # Commercial solver

---

## 2. Data Collection and Download

Download necessary data for the energy system model.

### 2.1 Geographic Data

Load or download geographic boundaries for Portugal.

In [3]:
# Download Portugal geographic boundaries
# Organized by country with ISO 3166-1 codes: PT (2-letter) or PRT (3-letter)

import requests
import io
import zipfile
from pathlib import Path

# Base dataset URL
base_url = "https://tubcloud.tu-berlin.de/s/567ckizz2Y6RLQq/download"

print("📍 Downloading Portugal geographic data from TU Berlin dataset...")
print("=" * 70)

# Portugal-specific data files to download
# ISO codes: PT (2-letter), PRT (3-letter)
portugal_files = {
    "GADM Administrative Boundaries": "gadm/gadm_410-levels-ADM_1-PRT.gpkg",
    "Copernicus Land Cover": "copernicus-glc/PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326-PT.tif",
    "GEBCO Elevation/Bathymetry": "gebco/GEBCO_2014_2D-PT.nc",
    "WDPA Protected Areas": "wdpa/WDPA_Oct2022_Public_shp-PRT.tif",
}

# Download individual files
portugal_data = {}
for data_type, file_path in portugal_files.items():
    try:
        print(f"\n📥 Downloading: {data_type}")
        print(f"   Path: {file_path}")
        
        # Construct download URL with file path parameter
        file_url = f"{base_url}?path=/{file_path}"
        
        response = requests.get(file_url, stream=True, timeout=60)
        response.raise_for_status()
        
        file_size_mb = len(response.content) / 1024 / 1024
        print(f"   Size: {file_size_mb:.2f} MB")
        
        # Save file maintaining directory structure
        local_path = DATA_RAW / file_path
        local_path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(local_path, 'wb') as f:
            f.write(response.content)
        
        portugal_data[data_type] = local_path
        print(f"   ✅ Saved to: {local_path}")
        
    except Exception as e:
        print(f"   ⚠️ Error downloading {data_type}: {e}")

print("\n" + "=" * 70)

# Load geographic data for Portugal (GADM boundaries)
print("\n📍 Loading Portugal geographic boundaries (GADM)...")

portugal_gdf = None

# Try GADM first (primary source for administrative boundaries)
gadm_file = DATA_RAW / "gadm" / "gadm_410-levels-ADM_1-PRT.gpkg"

if gadm_file.exists():
    try:
        print(f"   Loading GADM data from: {gadm_file.name}")
        portugal_gdf = gpd.read_file(gadm_file)
        print(f"   ✅ Loaded {len(portugal_gdf)} administrative regions from GADM")
        print(f"   Columns: {list(portugal_gdf.columns)}")
        
    except Exception as e:
        print(f"   ⚠️ Error reading GADM file: {e}")

# Display Portugal information
if portugal_gdf is not None and len(portugal_gdf) > 0:
    print(f"\n📊 Portugal Geographic Summary:")
    print(f"   - Data source: GADM v4.1.0 (Administrative Level 1)")
    print(f"   - Geometry type: {portugal_gdf.geometry.type.unique()}")
    print(f"   - CRS: {portugal_gdf.crs}")
    print(f"   - Number of regions: {len(portugal_gdf)}")
    
    # Show region names if available
    if 'NAME_1' in portugal_gdf.columns:
        print(f"   - Regions: {', '.join(portugal_gdf['NAME_1'].unique())}")
    
    print(f"\n✅ Geographic data loaded successfully!")
    
    # Save to processed data directory
    portugal_filepath = DATA_PROCESSED / "regions" / "portugal_gadm_boundaries.gpkg"
    portugal_filepath.parent.mkdir(parents=True, exist_ok=True)
    portugal_gdf.to_file(portugal_filepath, driver='GPKG')
    print(f"\n💾 Saved to: {portugal_filepath}")
    
    # Also save as shapefile for compatibility
    portugal_shp = DATA_PROCESSED / "regions" / "portugal_boundaries.shp"
    portugal_gdf.to_file(portugal_shp)
    print(f"💾 Also saved as: {portugal_shp}")
else:
    print("\n⚠️ Warning: Could not load GADM data")
    print("   Available files in data/raw:")
    for f in DATA_RAW.rglob("*"):
        if f.is_file():
            print(f"      • {f.relative_to(DATA_RAW)}")

# Display summary of downloaded data
print("\n" + "=" * 70)
print("📦 Downloaded Data Summary:")
for data_type, local_path in portugal_data.items():
    size_mb = local_path.stat().st_size / 1024 / 1024 if local_path.exists() else 0
    status = "✅" if local_path.exists() else "❌"
    print(f"   {status} {data_type}: {size_mb:.2f} MB")

print("\n✅ Portugal data download complete!")

# Display the boundaries
if portugal_gdf is not None:
    portugal_gdf

📍 Downloading Portugal geographic data from TU Berlin dataset...

📥 Downloading: GADM Administrative Boundaries
   Path: gadm/gadm_410-levels-ADM_1-PRT.gpkg
   Size: 8.71 MB
   ✅ Saved to: c:\Users\nashm\Documents\dsesm\group assignment\data\raw\gadm\gadm_410-levels-ADM_1-PRT.gpkg

📥 Downloading: Copernicus Land Cover
   Path: copernicus-glc/PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326-PT.tif
   Size: 1.84 MB
   ✅ Saved to: c:\Users\nashm\Documents\dsesm\group assignment\data\raw\copernicus-glc\PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326-PT.tif

📥 Downloading: GEBCO Elevation/Bathymetry
   Path: gebco/GEBCO_2014_2D-PT.nc
   Size: 1.56 MB
   ✅ Saved to: c:\Users\nashm\Documents\dsesm\group assignment\data\raw\gebco\GEBCO_2014_2D-PT.nc

📥 Downloading: WDPA Protected Areas
   Path: wdpa/WDPA_Oct2022_Public_shp-PRT.tif
   Size: 5.56 MB
   ✅ Saved to: c:\Users\nashm\Documents\dsesm\group assignment\data\raw\wdpa\WDPA_Oct2022_Public_

### 2.2.1 Load 2024 Historical Load Data

Load Portugal 2024 historical load data from local Excel file.

**Data Source:**
- **Platform:** Local Excel file (monthly_hourly_load_values_2024.xlsx)
- **Country:** Portugal (PT)
- **Resolution:** Hourly
- **Year:** 2024

**Requirements:**
1. ENTSO-E account credentials (username/email and password)
2. Secure environment variables (recommended):
   - `ENTSOE_USERNAME` - your ENTSO-E email/username
   - `ENTSOE_PASSWORD` - your ENTSO-E password

In [4]:
# Load 2024 Portugal electricity load data from local Excel file
# Source: Monthly-hourly resolution load data
# Data type: Actual load (measured consumption)
# Resolution: Hourly
# File: monthly_hourly_load_values_2024.xlsx

print("⚡ Loading 2024 electricity load data for Portugal from local file...")
print("=" * 70)

portugal_load_2024 = None

# Check for the Excel file
excel_file = DATA_PROCESSED / "load" / "monthly_hourly_load_values_2024.xlsx"

if excel_file.exists():
    print(f"\n📂 Found local load data file: {excel_file.name}")
    print(f"   File size: {excel_file.stat().st_size / 1024:.2f} KB")
    
    try:
        print(f"\n📥 Reading Excel file...")
        # Read the Excel file
        portugal_load_raw = pd.read_excel(excel_file)
        
        print(f"   ✅ Loaded {len(portugal_load_raw):,} records (all countries)")
        print(f"   📋 Columns: {list(portugal_load_raw.columns)}")
        
        # Filter for Portugal only (CountryCode = 'PT')
        if 'CountryCode' in portugal_load_raw.columns:
            portugal_load_raw = portugal_load_raw[portugal_load_raw['CountryCode'] == 'PT']
            print(f"\n🇵🇹 Filtered for Portugal (PT): {len(portugal_load_raw):,} records")
            
            if len(portugal_load_raw) == 0:
                print(f"   ⚠️ Warning: No data found for Portugal (PT)")
                print(f"   Available countries: {sorted(portugal_load_raw['CountryCode'].unique())}")
        else:
            print(f"   ℹ️ No CountryCode column found - assuming data is already for Portugal")
        
        # Display first few rows - only relevant columns
        print(f"\n🔍 First few rows of Portugal data (relevant columns):")
        display_cols = ['DateUTC', 'CountryCode', 'Value']
        print(portugal_load_raw[display_cols].head())
        
        # Check the actual year of the data
        data_year = pd.to_datetime(portugal_load_raw['DateUTC']).dt.year.iloc[0]
        print(f"\n📅 Data year detected: {data_year}")
        if data_year != YEAR:
            print(f"   ⚠️ WARNING: Data is from {data_year}, but model year is set to {YEAR}")
            print(f"   Using {data_year} data as proxy for {YEAR} model")

        
        # Process the data based on its structure
        # The Excel file has DateUTC and Value columns
        print(f"\n🔄 Processing Portugal data into timeseries format...")
        
        # Check if data has the expected columns from the Excel file
        if 'DateUTC' in portugal_load_raw.columns and 'Value' in portugal_load_raw.columns:
            # Use DateUTC as time and Value as load
            portugal_load_2024 = pd.DataFrame({
                'time': pd.to_datetime(portugal_load_raw['DateUTC']),
                'load_MW': portugal_load_raw['Value']
            })
            print(f"   ✅ Using DateUTC (actual timestamp) and Value columns")
            print(f"   ℹ️ Note: TimeFrom/TimeTo columns contain only time-of-day (not full dates)")
            
        elif 'time' in portugal_load_raw.columns or 'timestamp' in portugal_load_raw.columns or 'date' in portugal_load_raw.columns:
            # Already in timeseries format
            time_col = [col for col in portugal_load_raw.columns if col.lower() in ['time', 'timestamp', 'date', 'datetime', 'dateutc']][0]
            load_col = [col for col in portugal_load_raw.columns if 'load' in col.lower() or 'mw' in col.lower() or col.lower() in ['value', 'power']]
            
            if load_col:
                portugal_load_2024 = pd.DataFrame({
                    'time': pd.to_datetime(portugal_load_raw[time_col]),
                    'load_MW': portugal_load_raw[load_col[0]]
                })
            else:
                # Assume Value column contains load
                portugal_load_2024 = pd.DataFrame({
                    'time': pd.to_datetime(portugal_load_raw[time_col]),
                    'load_MW': portugal_load_raw['Value'] if 'Value' in portugal_load_raw.columns else portugal_load_raw[portugal_load_raw.columns[1]]
                })
        else:
            # Data might be in monthly-hourly matrix format
            # Try to reshape it into timeseries
            print(f"   ℹ️ Data appears to be in monthly-hourly matrix format")
            print(f"   🔄 Reshaping to timeseries...")
            
            # Create hourly timestamps for the data year
            start_date = pd.Timestamp(f'{YEAR}-01-01 00:00:00', tz='Europe/Lisbon')
            end_date = pd.Timestamp(f'{YEAR+1}-01-01 00:00:00', tz='Europe/Lisbon')
            timestamps = pd.date_range(start=start_date, end=end_date, freq='H', inclusive='left')
            
            # Flatten the data
            load_values = portugal_load_raw.values.flatten()
            
            # Match the lengths
            if len(load_values) >= len(timestamps):
                load_values = load_values[:len(timestamps)]
            else:
                print(f"   ⚠️ Warning: Data has fewer values ({len(load_values)}) than expected hours ({len(timestamps)})")
                # Pad with NaN if needed
                load_values = np.pad(load_values, (0, len(timestamps) - len(load_values)), constant_values=np.nan)
            
            portugal_load_2024 = pd.DataFrame({
                'time': timestamps,
                'load_MW': load_values
            })
            
            # Remove any NaN values
            portugal_load_2024 = portugal_load_2024.dropna()
        
        print(f"   ✅ Processed {len(portugal_load_2024):,} hourly data points")
        
        # Ensure time is datetime and remove timezone for consistency
        portugal_load_2024['time'] = pd.to_datetime(portugal_load_2024['time'])
        if portugal_load_2024['time'].dt.tz is not None:
            portugal_load_2024['time'] = portugal_load_2024['time'].dt.tz_localize(None)
        
        # Ensure load_MW is numeric
        portugal_load_2024['load_MW'] = pd.to_numeric(portugal_load_2024['load_MW'], errors='coerce')
        portugal_load_2024 = portugal_load_2024.dropna()
        
        print(f"\n{'='*70}")
        print(f"📊 PORTUGAL {YEAR} LOAD DATA ANALYSIS")
        print(f"{'='*70}")
        
        print(f"\n📋 Dataset Information:")
        print(f"   Shape: {portugal_load_2024.shape}")
        print(f"   Memory: {portugal_load_2024.memory_usage(deep=True).sum() / 1024:.2f} KB")
        
        # Temporal coverage
        print(f"\n📅 Temporal Coverage:")
        print(f"   - Start: {portugal_load_2024['time'].min()}")
        print(f"   - End: {portugal_load_2024['time'].max()}")
        print(f"   - Duration: {portugal_load_2024['time'].max() - portugal_load_2024['time'].min()}")
        print(f"   - Total hours: {len(portugal_load_2024):,}")
        
        # Check for missing data
        expected_hours = 8760 if YEAR % 4 != 0 else 8784  # Leap year check
        missing_hours = expected_hours - len(portugal_load_2024)
        if missing_hours != 0:
            print(f"   ⚠️ Missing {missing_hours} hours (expected {expected_hours})")
        else:
            print(f"   ✅ Complete dataset ({expected_hours} hours)")
        
        # Load statistics (in MW)
        print(f"\n📈 Load Statistics (MW):")
        print(f"   - Mean load: {portugal_load_2024['load_MW'].mean():.2f} MW")
        print(f"   - Median load: {portugal_load_2024['load_MW'].median():.2f} MW")
        print(f"   - Min load: {portugal_load_2024['load_MW'].min():.2f} MW")
        print(f"   - Max load: {portugal_load_2024['load_MW'].max():.2f} MW")
        print(f"   - Std deviation: {portugal_load_2024['load_MW'].std():.2f} MW")
        print(f"   - Load factor: {(portugal_load_2024['load_MW'].mean() / portugal_load_2024['load_MW'].max() * 100):.1f}%")
        
        # Energy statistics
        total_energy_MWh = portugal_load_2024['load_MW'].sum()
        total_energy_TWh = total_energy_MWh / 1e6
        print(f"\n⚡ Energy Statistics:")
        print(f"   - Total energy: {total_energy_MWh:,.0f} MWh")
        print(f"   - Total energy: {total_energy_TWh:.2f} TWh")
        print(f"   - Average daily: {total_energy_MWh / 366:.2f} MWh/day")  # 2024 is a leap year
        
        # Monthly statistics
        portugal_load_2024['month'] = pd.to_datetime(portugal_load_2024['time']).dt.month
        monthly_avg = portugal_load_2024.groupby('month')['load_MW'].agg(['mean', 'min', 'max'])
        
        print(f"\n📊 Monthly Statistics (MW):")
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        print(f"   {'Month':<8} {'Mean':>8} {'Min':>8} {'Max':>8}")
        print(f"   {'-'*8} {'-'*8} {'-'*8} {'-'*8}")
        for month, row in monthly_avg.iterrows():
            print(f"   {month_names[month-1]:<8} {row['mean']:>8.0f} {row['min']:>8.0f} {row['max']:>8.0f}")
        
        # Peak demand periods
        peak_load = portugal_load_2024.nlargest(10, 'load_MW')
        print(f"\n🔝 Top 10 Peak Demand Periods:")
        for idx, (_, row) in enumerate(peak_load.iterrows(), 1):
            timestamp = pd.to_datetime(row['time'])
            print(f"   {idx:2d}. {timestamp.strftime('%Y-%m-%d %H:%M')} - {row['load_MW']:,.0f} MW")
        
        # Low demand periods
        low_load = portugal_load_2024.nsmallest(10, 'load_MW')
        print(f"\n⬇️ Top 10 Low Demand Periods:")
        for idx, (_, row) in enumerate(low_load.iterrows(), 1):
            timestamp = pd.to_datetime(row['time'])
            print(f"   {idx:2d}. {timestamp.strftime('%Y-%m-%d %H:%M')} - {row['load_MW']:,.0f} MW")
        
        # Save processed load data for PyPSA
        processed_load_path = DATA_PROCESSED / "load" / f"portugal_load_{YEAR}_timeseries.csv"
        processed_load_path.parent.mkdir(parents=True, exist_ok=True)
        portugal_load_2024[['time', 'load_MW']].to_csv(processed_load_path, index=False)
        print(f"\n💾 Processed data saved to: {processed_load_path}")
        
        print(f"\n{'='*70}")
        print(f"✅ {YEAR} Load data successfully loaded and processed!")
        print(f"{'='*70}")
        
    except Exception as e:
        print(f"\n❌ Error reading Excel file: {e}")
        print(f"   Please check the file format and structure")
        portugal_load_2024 = None

else:
    print(f"\n❌ Excel file not found: {excel_file}")
    print(f"   Expected location: {excel_file}")
    print(f"\n   Please ensure the file exists at this location")
    print(f"   Or update the file path in the code above")

⚡ Loading 2024 electricity load data for Portugal from local file...

📂 Found local load data file: monthly_hourly_load_values_2024.xlsx
   File size: 16843.08 KB

📥 Reading Excel file...
   ✅ Loaded 313,978 records (all countries)
   📋 Columns: ['MeasureItem', 'DateUTC', 'DateShort', 'TimeFrom', 'TimeTo', 'CountryCode', 'Cov_ratio', 'Value', 'Value_ScaleTo100', 'CreateDate', 'UpdateDate']

🇵🇹 Filtered for Portugal (PT): 8,784 records

🔍 First few rows of Portugal data (relevant columns):
                   DateUTC CountryCode   Value
252558 2024-01-01 00:00:00          PT  5135.2
252559 2024-01-01 01:00:00          PT  4962.5
252560 2024-01-01 02:00:00          PT  4684.2
252561 2024-01-01 03:00:00          PT  4413.7
252562 2024-01-01 04:00:00          PT  4247.7

📅 Data year detected: 2024

🔄 Processing Portugal data into timeseries format...
   ✅ Using DateUTC (actual timestamp) and Value columns
   ℹ️ Note: TimeFrom/TimeTo columns contain only time-of-day (not full dates)
   ✅ Pro

### 2.3 Generation Capacity Data

Collect existing power plant and capacity data.

In [9]:
# Download and compile Portugal generation capacity data
# Sources: ENTSO-E, powerplantmatching, national statistics
# Data: Existing power plants by technology type for 2024

print("🏭 Collecting Portugal generation capacity data...")
print("=" * 70)

# Try using powerplantmatching library first
try:
    import powerplantmatching as pm
    print("\n📦 Using powerplantmatching library...")
    
    # Download power plant database for Portugal
    print("   Downloading global power plant database...")
    plants = pm.powerplants(from_url=True)
    
    # Filter for Portugal
    portugal_plants = plants[plants['Country'] == 'Portugal'].copy()
    
    if len(portugal_plants) > 0:
        print(f"\n✅ Found {len(portugal_plants)} power plants in Portugal")
        
        # Filter out decommissioned plants (plants with DateOut values)
        initial_count = len(portugal_plants)
        if 'DateOut' in portugal_plants.columns:
            portugal_plants = portugal_plants[portugal_plants['DateOut'].isna()].copy()
            decommissioned = initial_count - len(portugal_plants)
            print(f"   🔍 Filtered out {decommissioned} decommissioned plants")
            print(f"   ✅ {len(portugal_plants)} operational plants remaining")
        else:
            print(f"   ℹ️ No DateOut column found - assuming all plants are operational")
        
        # Add year of operation column (use commissioning year if available, else estimate)
        if 'YearCommissioned' in portugal_plants.columns:
            portugal_plants['Year_of_Operation'] = portugal_plants['YearCommissioned'].fillna(YEAR)
            print(f"   ℹ️ Using commissioning year data from database")
        else:
            portugal_plants['Year_of_Operation'] = YEAR
            print(f"   ℹ️ Using model year ({YEAR}) as year of operation")
        
        # Aggregate by technology type
        capacity_by_tech = portugal_plants.groupby('Fueltype')['Capacity'].sum().sort_values(ascending=False)
        
        print(f"\n📊 Installed Capacity by Technology (MW):")
        print(f"   {'Technology':<20} {'Capacity (MW)':>15}")
        print(f"   {'-'*20} {'-'*15}")
        for tech, capacity in capacity_by_tech.items():
            print(f"   {tech:<20} {capacity:>15,.0f}")
        
        print(f"\n   Total Capacity: {capacity_by_tech.sum():,.0f} MW")
        
        # Save detailed plant data
        plants_file = DATA_PROCESSED / "generation" / "portugal_power_plants.csv"
        plants_file.parent.mkdir(parents=True, exist_ok=True)
        portugal_plants.to_csv(plants_file, index=False)
        print(f"\n💾 Saved plant details to: {plants_file}")
        print(f"   Columns include: Year_of_Operation")
        
        # Save aggregated capacity data
        capacity_file = DATA_PROCESSED / "generation" / "portugal_capacity_by_technology.csv"
        capacity_by_tech.to_csv(capacity_file)
        print(f"💾 Saved capacity summary to: {capacity_file}")
        
    else:
        print("⚠️ No plants found in powerplantmatching database")
        raise ValueError("No data found")
        
except (ImportError, Exception) as e:
    if isinstance(e, ImportError):
        print("\n⚠️ powerplantmatching not installed")
    else:
        print(f"\n⚠️ Could not retrieve data from powerplantmatching: {e}")
    
    print("\n📝 Using reference data for Portugal (2024 estimates)")
    print("   Source: ENTSO-E, REN (Redes Energéticas Nacionais), IEA")
    
    # Portugal generation capacity data (approximate 2024 values in MW)
    portugal_capacity = {
        # Renewable Energy
        'Hydro': 7200,           # Includes pumped storage (~3.5 GW)
        'Wind Onshore': 5500,    # Strong wind sector
        'Solar PV': 2800,        # Growing rapidly
        'Biomass': 350,          # Biomass and waste
        
        # Conventional Generation
        'Natural Gas': 5100,     # CCGT and gas turbines
        'Coal': 0,               # Coal phase-out completed in 2021
        'Oil': 150,              # Backup/peaking units
        
        # Other
        'Other': 100             # Other technologies
    }
    
    print(f"\n📊 Portugal Installed Capacity by Technology (MW):")
    print(f"   {'Technology':<20} {'Capacity (MW)':>15}")
    print(f"   {'-'*20} {'-'*15}")
    
    total_capacity = 0
    for tech, capacity in portugal_capacity.items():
        print(f"   {tech:<20} {capacity:>15,.0f}")
        total_capacity += capacity
    
    print(f"   {'-'*20} {'-'*15}")
    print(f"   {'TOTAL':<20} {total_capacity:>15,.0f}")
    
    # Calculate renewable share
    renewable_capacity = portugal_capacity['Hydro'] + portugal_capacity['Wind Onshore'] + \
                        portugal_capacity['Solar PV'] + portugal_capacity['Biomass']
    renewable_share = (renewable_capacity / total_capacity) * 100
    
    print(f"\n🌱 Renewable Energy Statistics:")
    print(f"   - Renewable capacity: {renewable_capacity:,.0f} MW")
    print(f"   - Renewable share: {renewable_share:.1f}%")
    print(f"   - Conventional capacity: {total_capacity - renewable_capacity:,.0f} MW")
    
    # Create DataFrame for further use
    portugal_plants = pd.DataFrame([
        {'Technology': tech, 'Capacity_MW': cap, 'Fuel': tech, 'Status': 'Operating', 'Year_of_Operation': YEAR}
        for tech, cap in portugal_capacity.items()
    ])
    
    # Save to processed data
    capacity_file = DATA_PROCESSED / "generation" / "portugal_capacity_by_technology.csv"
    capacity_file.parent.mkdir(parents=True, exist_ok=True)
    portugal_plants.to_csv(capacity_file, index=False)
    print(f"\n💾 Saved capacity data to: {capacity_file}")
    print(f"   Columns include: Year_of_Operation (set to {YEAR})")

print("\n" + "=" * 70)
print("✅ Generation capacity data collection complete!")
print("=" * 70)

# Display summary DataFrame
print("\n📋 Capacity Summary:")
if 'portugal_plants' in locals():
    print(portugal_plants)

🏭 Collecting Portugal generation capacity data...

📦 Using powerplantmatching library...

✅ Found 415 power plants in Portugal
   🔍 Filtered out 11 decommissioned plants
   ✅ 404 operational plants remaining
   ℹ️ Using model year (2024) as year of operation

📊 Installed Capacity by Technology (MW):
   Technology             Capacity (MW)
   -------------------- ---------------
   Hydro                          9,164
   Wind                           5,231
   Natural Gas                    4,186
   Solar                          3,566
   Oil                              466
   Solid Biomass                    384
   Waste                             76
   Battery                           58
   Geothermal                        24
   Hydrogen Storage                   1
   Mechanical Storage                 1

   Total Capacity: 23,156 MW

⚠️ Could not retrieve data from powerplantmatching: [Errno 13] Permission denied: 'c:\\Users\\nashm\\Documents\\dsesm\\group assignment\\data\\proce

### 2.4 Weather Data for Renewable Resources

Use Atlite to download ERA5 weather data for renewable capacity factor calculations.

In [ ]:
# Download ERA5 weather data using Atlite for renewable capacity factor calculations
# Required for: Wind power and Solar PV capacity factors
# Data: ERA5 reanalysis (temperature, wind speed, solar radiation)

print("☀️ Downloading weather data for renewable resources...")
print("=" * 70)

try:
    import atlite
    print("\n📦 Using Atlite library for weather data...")
    print(f"   Atlite version: {atlite.__version__}")
    
    # Define cutout region for Portugal
    # Portugal bounding box: approximately 37°N-42°N, 9.5°W-6°W
    portugal_bounds = {
        'x': slice(-9.5, -6.0),    # Longitude (West to East)
        'y': slice(37.0, 42.0),     # Latitude (South to North)
        'time': f'{YEAR}'           # Full year
    }
    
    print(f"\n🗺️ Portugal geographic bounds:")
    print(f"   Latitude: {portugal_bounds['y'].start}°N to {portugal_bounds['y'].stop}°N")
    print(f"   Longitude: {portugal_bounds['x'].start}°W to {portugal_bounds['x'].stop}°W")
    print(f"   Time period: {YEAR}")
    
    # Create cutout path
    cutout_path = DATA_RAW / "weather" / f"portugal-{YEAR}.nc"
    cutout_path.parent.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📥 Preparing to download ERA5 data...")
    print(f"   Target file: {cutout_path.name}")
    print(f"   Resolution: Hourly")
    
    # Check if cutout already exists
    if cutout_path.exists():
        print(f"\n✅ Weather data already exists: {cutout_path.name}")
        print(f"   File size: {cutout_path.stat().st_size / 1024 / 1024:.2f} MB")
        print(f"   Loading existing cutout...")
        
        cutout = atlite.Cutout(path=cutout_path)
        print(f"   ✅ Loaded existing cutout")
    else:
        print(f"\n⚠️ CDS API required for ERA5 download")
        print(f"   Atlite needs CDS API credentials to download ERA5 data")
        print(f"\n   Setup instructions:")
        print(f"   1. Register at: https://cds.climate.copernicus.eu/")
        print(f"   2. Get your API key from your profile")
        print(f"   3. Create ~/.cdsapirc file with:")
        print(f"      url: https://cds.climate.copernicus.eu/api/v2")
        print(f"      key: <your-uid>:<your-api-key>")
        print(f"\n   Attempting download (will fail if not configured)...")
        
        # Create cutout and download ERA5 data
        cutout = atlite.Cutout(
            path=cutout_path,
            module='era5',
            bounds=portugal_bounds,
            time=f'{YEAR}'
        )
        
        # Download the data
        print(f"\n   Downloading ERA5 data... (this may take 10-30 minutes)")
        cutout.prepare()
        
        print(f"\n   ✅ Download complete!")
        print(f"   File size: {cutout_path.stat().st_size / 1024 / 1024:.2f} MB")
    
    # Display cutout information
    print(f"\n📊 Weather Data Summary:")
    print(f"   Coordinates: {cutout.coords}")
    print(f"   Available variables: {list(cutout.data.data_vars)}")
    print(f"   Time range: {cutout.data.time.values[0]} to {cutout.data.time.values[-1]}")
    print(f"   Spatial resolution: ~{abs(cutout.dx):.2f}° x {abs(cutout.dy):.2f}°")
    
    # Save cutout reference for later use
    weather_data = {
        'cutout': cutout,
        'path': cutout_path,
        'bounds': portugal_bounds
    }
    
    print(f"\n✅ Weather data ready for capacity factor calculations!")
    
except ImportError:
    print("\n⚠️ Atlite not installed")
    print("   Install with: conda install -c conda-forge atlite")
    print("   Or: pip install atlite")
    weather_data = None
    
except Exception as e:
    print(f"\n❌ Error with weather data: {e}")
    print("\n💡 Alternative approach:")
    print("   - Use pre-calculated capacity factors from literature")
    print("   - Download from EMHIRES database (EU Joint Research Centre)")
    print("   - Use PyPSA-EUR capacity factor data")
    print("\n   For now, we'll use typical capacity factors in the model:")
    print("   - Wind onshore: 25-30% capacity factor")
    print("   - Solar PV: 15-20% capacity factor")
    print("   - These will be applied in the modeling section")
    
    weather_data = None

print("\n" + "=" * 70)
print("✅ Weather data section complete!")
print("=" * 70)

# Display summary
if weather_data:
    print("\n📋 Weather data successfully loaded and ready for use")
else:
    print("\n📋 Using typical capacity factors (no weather data downloaded)")

---

## 3. Data Processing and Preparation

Clean and prepare all data for modeling.

### 3.1 Process Geographic Eligibility

Determine eligible areas for renewable energy deployment.

In [ ]:
# TODO: Process eligibility data
# Exclusions:
# - Protected areas
# - Urban areas
# - Water bodies
# - Steep slopes for wind/solar

# Placeholder
print("🗺️ Eligibility processing to be implemented")

### 3.2 Calculate Renewable Capacity Factors

Use Atlite to calculate capacity factors for wind and solar.

In [ ]:
# TODO: Calculate capacity factors using Atlite
# Technologies:
# - Onshore wind
# - Solar PV
# - Offshore wind (optional)

# Placeholder
print("🌬️ Capacity factor calculation to be implemented")

### 3.3 Process Load Profiles

Clean and format electricity demand time series.

In [ ]:
# TODO: Process load data
# Steps:
# - Handle missing values
# - Resample to hourly
# - Validate total consumption

# Placeholder
print("📊 Load profile processing to be implemented")

---

## 4. PyPSA Network Building

Construct the energy system model.

### 4.1 Initialize Network

Create the base PyPSA network object.

In [ ]:
# TODO: Initialize PyPSA network
# Create network with snapshots

# Placeholder
print("🔌 Network initialization to be implemented")

### 4.2 Add Buses (Nodes)

Define the network nodes/buses.

In [ ]:
# TODO: Add buses to network
# Options:
# - Single node (copper plate)
# - Regional nodes
# - Detailed network topology

# Placeholder
print("📍 Bus addition to be implemented")

### 4.3 Add Generators

Add generation technologies and capacities.

In [ ]:
# TODO: Add generators to network
# Technologies:
# - Solar PV
# - Onshore wind
# - Hydro (run-of-river and reservoir)
# - Natural gas
# - Coal (if applicable)

# Parameters:
# - p_nom (nominal capacity)
# - p_nom_extendable (allow capacity expansion)
# - marginal_cost
# - capital_cost
# - efficiency
# - carrier (technology type)

# Placeholder
print("⚡ Generator addition to be implemented")

### 4.4 Add Loads

Add electricity demand to buses.

In [ ]:
# TODO: Add loads to network
# Use processed demand time series

# Placeholder
print("📊 Load addition to be implemented")

### 4.5 Add Transmission Lines (Optional)

Add transmission infrastructure if using multi-node model.

In [ ]:
# TODO: Add transmission lines
# Parameters:
# - s_nom (capacity)
# - x (reactance)
# - r (resistance)
# - length

# Placeholder
print("🔌 Transmission line addition to be implemented (if multi-node)")

### 4.6 Add Storage (Optional)

Add battery or other storage technologies.

In [ ]:
# TODO: Add storage units
# Types:
# - Battery storage
# - Pumped hydro storage

# Placeholder
print("🔋 Storage addition to be implemented (optional)")

### 4.7 Verify Network

Check network consistency and display summary.

In [ ]:
# TODO: Verify and display network
# Check:
# - All buses have loads or generators
# - No isolated components
# - Capacity factors properly assigned
# - Cost parameters reasonable

# Placeholder
print("✅ Network verification to be implemented")

---

## 5. Model Optimization

Run the optimization to find optimal dispatch and/or capacity expansion.

### 5.1 Run Optimization

Execute the PyPSA optimization.

In [ ]:
# TODO: Run optimization
# network.optimize(solver_name=SOLVER)

# Placeholder
print("⚡ Optimization execution to be implemented")

### 5.2 Check Optimization Status

Verify that optimization converged successfully.

In [ ]:
# TODO: Check optimization status
# Print objective value and solver status

# Placeholder
print("✅ Optimization status check to be implemented")

---

## 6. Results Analysis and Visualization

Analyze and visualize the optimization results.

### 6.1 System Costs

Calculate and display total system costs.

In [ ]:
# TODO: Calculate system costs
# - Total objective value
# - Breakdown by component (capital, operation, fuel)
# - Cost per MWh

# Placeholder
print("💰 System cost analysis to be implemented")

### 6.2 Generation Mix

Analyze the electricity generation mix.

In [ ]:
# TODO: Analyze generation mix
# - Total generation by technology
# - Capacity factors
# - Annual energy production

# Placeholder
print("⚡ Generation mix analysis to be implemented")

### 6.3 Capacity Expansion Results

Display optimal capacity investments (if capacity expansion was allowed).

In [ ]:
# TODO: Analyze capacity expansion
# - Optimal additions by technology
# - Comparison to existing capacity

# Placeholder
print("🏗️ Capacity expansion analysis to be implemented")

### 6.4 Time Series Visualization

Plot generation and demand over time.

In [ ]:
# TODO: Create time series plots
# - Dispatch stack plot
# - Load vs generation
# - Renewable curtailment (if any)
# - Storage operation

# Placeholder
print("📈 Time series visualization to be implemented")

### 6.5 Emissions Analysis

Calculate CO₂ emissions.

In [ ]:
# TODO: Calculate emissions
# - Total CO₂ emissions
# - Emissions intensity (gCO₂/kWh)
# - Breakdown by fuel type

# Placeholder
print("🌍 Emissions analysis to be implemented")

### 6.6 Geographic Visualization

Create maps showing capacity distribution (if multi-node model).

In [ ]:
# TODO: Create geographic maps
# - Network topology
# - Generator locations and sizes
# - Transmission lines
# - Renewable resource potential

# Placeholder
print("🗺️ Geographic visualization to be implemented")

---

## 7. Scenario Analysis (Optional)

Compare different scenarios or sensitivity analysis.

### 7.1 Define Scenarios

Define alternative scenarios to compare (e.g., different CO₂ constraints, cost assumptions).

In [ ]:
# TODO: Define scenarios
# Examples:
# - Baseline
# - High renewable target
# - CO₂ cap
# - No nuclear/coal
# - Different cost assumptions

# Placeholder
print("🎯 Scenario definition to be implemented (optional)")

### 7.2 Run Scenarios

Execute optimization for each scenario.

In [ ]:
# TODO: Run scenario loop
# For each scenario, optimize and store results

# Placeholder
print("🔄 Scenario execution to be implemented (optional)")

### 7.3 Compare Scenarios

Create comparison plots and tables.

In [ ]:
# TODO: Compare scenario results
# - Cost comparison
# - Generation mix comparison
# - Emissions comparison
# - Capacity comparison

# Placeholder
print("📊 Scenario comparison to be implemented (optional)")

---

## 8. Conclusions and Summary

Summarize key findings and insights.

### Key Findings

TODO: Summarize main results:
- Optimal generation mix
- Total system costs
- CO₂ emissions levels
- Renewable energy share
- Key bottlenecks or constraints
- Policy recommendations

### Limitations and Future Work

TODO: Discuss:
- Model assumptions and their impact
- Data limitations
- Suggested improvements
- Additional scenarios to explore

---

## 9. Export Results

Save results and figures for reporting.

In [ ]:
# TODO: Save results
# - Export network to NetCDF
# - Save key metrics to CSV
# - Export figures

# Placeholder
print("💾 Results export to be implemented")

In [ ]:
# TODO: Create geographic maps
# - Network topology
# - Generator locations and sizes
# - Transmission lines
# - Renewable resource potential

# Placeholder
print("🗺️ Geographic visualization to be implemented")